# to_dict Function Documentation

The [to_dict](cci:1://file:///Users/lion/untitled%20folder/lion-os/tests/test_libs_data_handlers/test_to_dict.py:17:4-18:31) function is a versatile utility that converts various Python objects into dictionary format. It supports a wide range of input types and provides flexible configuration options for handling different scenarios.

## Key Features

- **Type Flexibility**: Handles various Python types including None, strings (JSON/XML), custom models, enums, and more
- **Recursive Conversion**: Optional deep conversion of nested objects
- **Custom Parser Support**: Allows custom parsing logic for special cases
- **Error Handling**: Built-in error suppression option
- **Model Integration**: Special handling for Pydantic and custom model classes

## Function Signature

```python
def to_dict(
    input_: Any,
    use_model_dump: bool = True,
    fuzzy_parse: bool = False,
    suppress: bool = False,
    str_type: Literal["json", "xml"] | None = "json",
    parser: Callable[[str], Any] | None = None,
    recursive: bool = False,
    max_recursive_depth: int = None,
    recursive_python_only: bool = True,
    use_enum_values: bool = False,
    **kwargs: Any,
) -> dict

In [1]:
from lion.libs.parse import to_dict

## Type-Specific Behavior

The function handles different types with specialized logic:

### 1. NoneType

In [2]:
from pydantic_core import PydanticUndefined
from lion.utils import UNDEFINED

print(to_dict(None))
print(to_dict(PydanticUndefined))
print(to_dict(UNDEFINED))

{}
{}
{}


### String Handling

The function can parse both JSON and XML strings:
- **JSON Mode**: Default parser that handles both well-formed and malformed JSON (with fuzzy parsing)
- **XML Mode**: Converts XML strings to dictionary format with optional root tag removal

### 2. String

In [3]:
# proper json

json_str = '{"a": 1, "b": 2, "c": 3}'
to_dict(json_str)

{'a': 1, 'b': 2, 'c': 3}

In [4]:
# partial json

json_str1 = '{"name": "lion", "age": 3'
to_dict(json_str1, fuzzy_parse=True)

{'name': 'lion', 'age': 3}

In [5]:
# proper xml

xml_str = "<a><b>1</b><c>2</c></a>"
print(to_dict(xml_str, str_type="xml"))

{'a': {'b': '1', 'c': '2'}}


In [6]:
# remvoe xml root tag

print(to_dict(xml_str, str_type="xml"))

{'a': {'b': '1', 'c': '2'}}


### Model Conversion

For custom model classes, the function attempts to use built-in serialization methods in the following order:
1. model_dump()
2. to_dict()
3. `to_json()`
4. dict()
5. json()

The `use_model_dump` parameter controls whether to prioritize using model_dump() or fall back to other methods.

### 3. Custom Models

we check for the following methods:
- `model_dump`
- `to_dict`
- `to_json`
- `dict`
- `json`

In [7]:
from lion.protocols.component import Component

a = Component()

In [8]:
# default is model_dump method

print(to_dict(a).keys())

dict_keys(['id', 'created_timestamp', 'metadata', 'embedding'])


In [9]:
# using the model's custom methods instead of model_dump

print(to_dict(a, use_model_dump=False).keys())

dict_keys(['id', 'created_timestamp', 'metadata', 'embedding', 'lion_class'])


### Recursive Conversion

The recursive mode allows deep conversion of nested objects:
- Controls depth with `max_recursive_depth` (default: 5, max: 10)
- Option to only convert Python built-in types (`recursive_python_only=True`)
- Handles circular references and complex object hierarchies

### 4. Recursive to_dict

In [10]:
from pydantic import BaseModel


class NestedModel(BaseModel):
    a: Component
    b: Component


nested = {"a": NestedModel(a=Component(), b=Component())}

In [11]:
to_dict(nested)

{'a': NestedModel(a=Component(id=IDType('9ba8c7966d722e0cfd0c64fc230b3814'), created_timestamp=1734042137.588441, metadata={}, embedding=[]), b=Component(id=IDType('f7acac98689653fbb1fd93dd36b573bf'), created_timestamp=1734042137.58849, metadata={}, embedding=[]))}

In [12]:
to_dict(nested, recursive=True, recursive_python_only=False)

{'a': {'a': {'id': '9ba8c7966d722e0cfd0c64fc230b3814',
   'created_timestamp': 1734042137.588441,
   'metadata': {},
   'embedding': []},
  'b': {'id': 'f7acac98689653fbb1fd93dd36b573bf',
   'created_timestamp': 1734042137.58849,
   'metadata': {},
   'embedding': []}}}

### Enum Handling

Special handling for Python's Enum types:
- Can preserve enum members as enum objects or convert to their values
- Handles nested enums with recursive conversion
- Supports None and UNDEFINED as valid enum values

### 5. Enum

In [13]:
from enum import Enum
from lion.utils import UNDEFINED


class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
    ORANGE = 3
    PURPLE = None
    YELLOW = UNDEFINED


to_dict(Color)

{'RED': <Color.RED: 1>,
 'GREEN': <Color.GREEN: 2>,
 'BLUE': <Color.BLUE: 3>,
 'ORANGE': <Color.BLUE: 3>,
 'PURPLE': <Color.PURPLE: None>,
 'YELLOW': <Color.YELLOW: UNDEFINED>}

In [14]:
# use enum values instead of names

to_dict(Color, use_enum_values=True)

{'RED': 1,
 'GREEN': 2,
 'BLUE': 3,
 'ORANGE': 3,
 'PURPLE': None,
 'YELLOW': UNDEFINED}

In [15]:
class BrightColor(Enum):
    DARK = Color
    BRIGHT = Color


to_dict(BrightColor)

{'DARK': <BrightColor.DARK: <enum 'Color'>>,
 'BRIGHT': <BrightColor.DARK: <enum 'Color'>>}

In [16]:
to_dict(BrightColor, use_enum_values=True)

{'DARK': <enum 'Color'>, 'BRIGHT': <enum 'Color'>}

In [17]:
# recursive enum values

to_dict(
    BrightColor,
    recursive=True,
    use_enum_values=True,
)

{'DARK': {'RED': 1,
  'GREEN': 2,
  'BLUE': 3,
  'ORANGE': 3,
  'PURPLE': None,
  'YELLOW': UNDEFINED},
 'BRIGHT': {'RED': 1,
  'GREEN': 2,
  'BLUE': 3,
  'ORANGE': 3,
  'PURPLE': None,
  'YELLOW': UNDEFINED}}

### Other Types

The function provides consistent handling for various Python collections:
- **Sets**: Converted to dictionaries with elements as both keys and values
- **Iterables**: Converted to dictionaries with index-based keys
- **Mappings**: Preserved as-is with potential recursive conversion of values

### 6. Others

In [18]:
# set (key and value are the same)

from lion.libs.parse import to_dict


a = {1, "a", "c", 5}
to_dict(a)

{'a': 'a', 1: 1, 5: 5, 'c': 'c'}

In [19]:
# Iterable (keys are numbers starting from 0)

a = [1, "a", "c", 5]
to_dict(a)

{0: 1, 1: 'a', 2: 'c', 3: 5}